<a href="https://colab.research.google.com/github/dolevi101/PPKMeans/blob/main/Privacy_Preserving_K_Means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tenseal
!pip install numpy
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 25.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import copy
import tenseal as ts
import time
import numpy as np
import math
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import itertools
import random

In [26]:
def max_with_lp(distances, p=20):
  #alpha = ts.plain_tensor(1/p)
  alpha = 1/p
  #return taylor_series_sqrt_x_plus_1_alpha((x1**p + x2**p+ x3**p+ x4**p) - 1, alpha)
  result_pow = [dist**p for dist in distances]
  return sum(result_pow)
  #return (x1**p + x2**p+ x3**p+ x4**p)


def taylor_series_sqrt_x_plus_1_alpha(x, a):
  sum_elements = 1
  #print("a={}, x={}".format(a,x.data))
  sum_elements += x*a
  sum_elements += 0.5*a*(a-1)*(x**2)
  sum_elements += (1/6)*a*(a-1)*(a-2)*(x**3)
  return sum_elements


def distance(a, b):
  return a-b


def get_p_value(x, max_val, p,n):
  #approx = approx_to_1_div_x(max_val,n)
  approx = inverse(max_val,n)
  xp = x**p
  #print("xp = {} approx = {}, real = {}".format(xp.decrypt().tolist()[0], approx.decrypt().tolist()[0], (1/(max_val.decrypt().tolist()[0]))))
  #print("x = {}, max_val = {}".format(x, max_val))
  #return xp/max_val
  prob = xp * approx
  return prob
  

#converges only when 0<x<2
def approx_to_1_div_x(x, n):
  #print("x in approx_to_1_div_x is: {}".format(x))
  mul = 1
  for i in range(n):
    mul *= (1+(1-x)**(2**i))
  #print("1/{} = {}".format(x,mul))
  return mul


def inverse(x, n):
    a = 2 - x
    b = 1 - x
    for i in range(n):
        b = b**2
        a = a*(1 + b)
    return a


def prob_to_be_in_center(data, centers, p,n):
  num_of_centers = len(centers)
  num_of_points = len(data)
  distances = []
  sum_distance_p_from_center = []
  for x in data:
      temp_dis = []
      temp_sum = 0
      for c in centers:
            dis = distance(x, c)
            temp_dis.append(dis)
            dis_pow = dis**p
            temp_sum += dis_pow

      sum_distance_p_from_center.append(temp_sum)
      distances.append(temp_dis)
    

  probs = []
  p_values_sum = []
  for i in range(num_of_centers):
    probs_sum = 0
    p_values_sum_per_center = 0
    for j in range(num_of_points):
        p_value = get_p_value(distances[j][i], sum_distance_p_from_center[j], p,n)
        probs_sum += data[j] * (1 - p_value)
        p_values_sum_per_center += (1 - p_value)
        if to_print:
          print("@@@@@")
          if is_pp_case:
            print("point: {}, center: {}, 1 - p_value: {}".format(data[j].decrypt().tolist()[0], centers[i].decrypt().tolist()[0], 1 - p_value.decrypt().tolist()[0])) 
          else:
            print("point: {}, center: {}, 1 - p_value: {}".format(data[j], centers[i], 1 - p_value))
    probs.append(probs_sum)
    p_values_sum.append(p_values_sum_per_center)
    print("finished center {}/{}".format(i+1, num_of_centers))
  return probs, p_values_sum


def get_ckks_context():
  # parameters
  #poly_mod_degree = 2**12
  poly_mod_degree = 2**13
  #coeff_mod_bit_sizes = [40, 20, 40]
  coeff_mod_bit_sizes = [35, 25, 25, 25, 25, 25, 25, 25, 25, 35]
  # create TenSEALContext
  context = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
  # scale of ciphertext to use
  #context.global_scale = 2 ** 20
  context.global_scale = 2 ** 25
  # this key is needed for doing dot-product operations
  context.generate_galois_keys()
  return context


def get_other_context():
  # parameters
  #poly_mod_degree = 8192
  poly_mod_degree = 16384
  #poly_mod_degree = 32768
  #coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 21, 21, 40]
  coeff_mod_bit_sizes = [60,35,35,35,35,35,35,35,35,35,60]
  #coeff_mod_bit_sizes = [60,50,50,50,50,50,50,50,50,50,50,50,60]
  print("sum of coeff_mod_bit_sizes=", sum(coeff_mod_bit_sizes))
  # create TenSEALContext
  ctx_training = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
  ctx_training.global_scale = 2 ** 35
  ctx_training.generate_galois_keys()
  return ctx_training


def print_fig(X, centers_colors, plot_name):
    # Create a scatter plot
    fig, axs = plt.subplots(nrows=1, ncols=1)
    axs.scatter(X, [0]*len(X), c=centers_colors)
    axs.set_title(plot_name)
    fig.show()

In [5]:
def real_kmeans(n_clusters, data):
    print("real_kmeans...")
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(np.array(data).reshape(-1, 1))
    labels_data = [0]*len(data)
    labels_centers = [1]*len(kmeans.cluster_centers_)
    labels = labels_data + labels_centers
    X = data + list(kmeans.cluster_centers_)  
    print_fig(X, labels, "real kmeans")
    return kmeans.cluster_centers_

In [28]:
def our_kmeans(data, centers, n, p, iterations):
    is_pp_case = False
    print("our_kmeans...")
    start = time.time()
    probs, p_values_sum = prob_to_be_in_center(data, centers,p,n)
    new_centers = [probs[c_index]/p_values_sum[c_index] for c_index in range(len(centers))]
    end = time.time()

    print("({}/{}) centers = {}, Duration: {} seconds".format(1, iterations, new_centers, end - start))

    for i in range(1, iterations):
        start = time.time()
        probs, p_values_sum = prob_to_be_in_center(data, new_centers,p,n)
        new_centers = [probs[c_index]/p_values_sum[c_index] for c_index in range(len(centers))]
        end = time.time()
        print("({}/{}) centers = {}, Duration: {} seconds".format(i+1, iterations, new_centers, end - start))

    print("({}/{}) centers = {}, Duration: {} seconds".format(i+1, iterations, new_centers, end - start))

    labels_data = [0]*len(data)
    labels_centers = [1]*len(new_centers)
    labels = labels_data + labels_centers
    X = data + new_centers    
    print_fig(X, labels, "our kmeans")

In [29]:
def encrypted_kmeans(data, centers, n, p, iterations):
    is_pp_case = True
    print("encrypted_kmeans...")
    context = get_other_context()
    enc_data = [ts.ckks_tensor(context, [item]) for item in data]
    enc_centers = [ts.ckks_tensor(context, [item]) for item in centers]
    
    start = time.time()
    res_probs, res_p_values_sum = prob_to_be_in_center(enc_data, enc_centers,p,n)
    probs, p_values_sum = [item.decrypt().tolist()[0] for item in res_probs], [item.decrypt().tolist()[0] for item in res_p_values_sum]
    
    new_centers = [probs[c_index]/p_values_sum[c_index] for c_index in range(len(centers))]
    end = time.time()

    print("({}/{}) centers = {}, Duration: {} seconds".format(1, iterations, new_centers, end - start))

    for i in range(1, iterations):
        start = time.time()
        enc_data = [ts.ckks_tensor(context, [item]) for item in data]
        enc_centers = [ts.ckks_tensor(context, [item]) for item in new_centers]
        res_probs, res_p_values_sum = prob_to_be_in_center(enc_data, enc_centers,p,n)
        probs, p_values_sum = [item.decrypt().tolist()[0] for item in res_probs], [item.decrypt().tolist()[0] for item in res_p_values_sum]
        new_centers = [probs[c_index]/p_values_sum[c_index] for c_index in range(len(centers))]
        end = time.time()
        print("({}/{}) centers = {}, Duration: {} seconds".format(i+1, iterations, new_centers, end - start))

    print("({}/{}) centers = {}, Duration: {} seconds".format(i+1, iterations, new_centers, end - start))

    labels_data = [0]*len(data)
    labels_centers = [1]*len(new_centers)
    labels = labels_data + labels_centers
    X = data + new_centers    
    print_fig(X, labels, "encrypted kmeans")

In [7]:
static_data = [random.uniform(0,0.2) for i in range(50)] + [random.uniform(0.5,0.9) for i in range(50)]

In [30]:
x1 = 0.1
x2 = 0.9
x3 = 0.2
x4 = 0.8
data = [x1, x2, x3, x4]
c1 = 0.48
c2 = 0.69
initial_centers = [c1, c2]

n=3
#p must be even
p=2
iterations = 9
to_print = True

#data = static_data

#our_kmeans(data, initial_centers, n, p, iterations)
#real_kmeans(len(initial_centers), data)
encrypted_kmeans(data, initial_centers, n, p, iterations)

encrypted_kmeans...
sum of coeff_mod_bit_sizes= 435
@@@@@
point: 0.1000001748596818, center: 0.48000007852831283, 1 - p_value: 0.7067416699232083
@@@@@
point: 0.8999999318301326, center: 0.48000007852831283, 1 - p_value: 0.21465930344549833
@@@@@
point: 0.20000006013387092, center: 0.48000007852831283, 1 - p_value: 0.7543200068921464
@@@@@
point: 0.8000000256135943, center: 0.48000007852831283, 1 - p_value: 0.23324296615707563
finished center 1/2
@@@@@
point: 0.1000001748596818, center: 0.6900001293482604, 1 - p_value: 0.29305212489423094
@@@@@
point: 0.8999999318301326, center: 0.6900001293482604, 1 - p_value: 0.8036650380344883
@@@@@
point: 0.20000006013387092, center: 0.6900001293482604, 1 - p_value: 0.24760426385090772
@@@@@
point: 0.8000000256135943, center: 0.6900001293482604, 1 - p_value: 0.9093964847206911
finished center 2/2
(1/9) centers = [0.31502535297303064, 0.6787711780359035], Duration: 155.4796965122223 seconds
@@@@@
point: 0.10000011417298893, center: 0.315025537644676

KeyboardInterrupt: ignored